# Self-Consistency

* **Ideia**: ensaio de múltiplos caminhos de raciocínio seguidos por votação majoritária sobre a resposta final
* **Estratégia:** problemas que têm uma resposta correta única tendem a admitir vários caminhos de raciocínio válidos que convergem para essa mesma resposta

In [1]:
# Imports necessários
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from pathlib import Path
import os
import re
import time 
from dotenv import load_dotenv
import collections

In [2]:
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

### Ajustando a temperatura do modelo para que exista variabilidade das respostas

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

## Verificação se a inteface está funcionando

In [4]:
prompt = """
Olá, tudo bem?
"""

response = llm.invoke(prompt)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Resposta:*

----

Olá! Estou bem, obrigado. E você, como está?

----

## Aqui considero que a conversa **não** tem memória

In [5]:
prompt = """
O que você disse?
"""

response = llm.invoke(prompt)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Resposta:*

----

Eu disse que sou um modelo de linguagem treinado para ajudar com perguntas e fornecer informações. Como posso ajudar você hoje?

----

## **PROMPT 01:** Zero-shot

In [6]:
prompt = """
Q: Um cliente compra 12 unidades de um produto cujo preço
unitário é R 125,80. Antes dos impostos há um desconto de 5 %
sobre o subtotal. Depois do desconto aplicam-se dois impostos:
ICMS de 18 % e PIS de 1,65 %, ambos sobre o subtotal já com
desconto. Há ainda uma taxa fixa de frete de R 48,00.
Qual é o valor total a pagar na nota fiscal?
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

response = llm.invoke(prompt)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Prompt:*

----


Q: Um cliente compra 12 unidades de um produto cujo preço
unitário é R 125,80. Antes dos impostos há um desconto de 5 %
sobre o subtotal. Depois do desconto aplicam-se dois impostos:
ICMS de 18 % e PIS de 1,65 %, ambos sobre o subtotal já com
desconto. Há ainda uma taxa fixa de frete de R 48,00.
Qual é o valor total a pagar na nota fiscal?


----

*Resposta:*

----

Vamos calcular o valor total a pagar na nota fiscal passo a passo.

1. **Cálculo do subtotal:**
   - Preço unitário: R$ 125,80
   - Quantidade: 12 unidades
   - Subtotal = Preço unitário × Quantidade = 125,80 × 12 = R$ 1.509,60

2. **Cálculo do desconto:**
   - Desconto de 5% sobre o subtotal:
   - Desconto = 5% de R$ 1.509,60 = 0,05 × 1.509,60 = R$ 75,48

3. **Cálculo do subtotal com desconto:**
   - Subtotal com desconto = Subtotal - Desconto = 1.509,60 - 75,48 = R$ 1.434,12

4. **Cálculo dos impostos:**
   - ICMS de 18% sobre o subtotal com desconto:
   - ICMS = 18% de R$ 1.434,12 = 0,18 × 1.434,12 = R$ 258,14 (aproximadamente)

   - PIS de 1,65% sobre o subtotal com desconto:
   - PIS = 1,65% de R$ 1.434,12 = 0,0165 × 1.434,12 = R$ 23,64 (aproximadamente)

5. **Cálculo do total dos impostos:**
   - Total de impostos = ICMS + PIS = 258,14 + 23,64 = R$ 281,78

6. **Cálculo do valor total a pagar:**
   - Valor total a pagar = Subtotal com desconto + Total de impostos + Frete
   - Frete = R$ 48,00
   - Valor total = 1.434,12 + 281,78 + 48,00 = R$ 1.763,90

Portanto, o valor total a pagar na nota fiscal é **R$ 1.763,90**.

----

## **PROMPT 02:** Aqui vou fazer um prompt Cot

In [7]:
prompt = """
Q: Um cliente compra 12 unidades de um produto cujo preço
unitário é R 125,80. Antes dos impostos há um desconto de 5 %
sobre o subtotal. Depois do desconto aplicam-se dois impostos:
ICMS de 18 % e PIS de 1,65 %, ambos sobre o subtotal já com
desconto. Há ainda uma taxa fixa de frete de R 48,00.
Qual é o valor total a pagar na nota fiscal?

Vamos pensar passo a passo.

Quando terminar, escreva a resposta final exatamente no formato:

Answer: <valor em reais com 2 casas decimais>
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

response = llm.invoke(prompt)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Prompt:*

----


Q: Um cliente compra 12 unidades de um produto cujo preço
unitário é R 125,80. Antes dos impostos há um desconto de 5 %
sobre o subtotal. Depois do desconto aplicam-se dois impostos:
ICMS de 18 % e PIS de 1,65 %, ambos sobre o subtotal já com
desconto. Há ainda uma taxa fixa de frete de R 48,00.
Qual é o valor total a pagar na nota fiscal?

Vamos pensar passo a passo.

Quando terminar, escreva a resposta final exatamente no formato:

Answer: <valor em reais com 2 casas decimais>


----

*Resposta:*

----

Vamos calcular passo a passo o valor total a pagar na nota fiscal.

1. **Cálculo do subtotal**:
   Preço unitário = R$ 125,80  
   Quantidade = 12 unidades  
   Subtotal = Preço unitário × Quantidade  
   Subtotal = 125,80 × 12 = R$ 1.509,60  

2. **Cálculo do desconto**:
   Desconto = 5% do subtotal  
   Desconto = 0,05 × 1.509,60 = R$ 75,48  

3. **Cálculo do subtotal com desconto**:
   Subtotal com desconto = Subtotal - Desconto  
   Subtotal com desconto = 1.509,60 - 75,48 = R$ 1.434,12  

4. **Cálculo dos impostos**:
   - ICMS = 18% do subtotal com desconto  
     ICMS = 0,18 × 1.434,12 = R$ 258,14  
     
   - PIS = 1,65% do subtotal com desconto  
     PIS = 0,0165 × 1.434,12 = R$ 23,66  

5. **Cálculo do total de impostos**:
   Total de impostos = ICMS + PIS  
   Total de impostos = 258,14 + 23,66 = R$ 281,80  

6. **Cálculo do total a pagar**:
   Total a pagar = Subtotal com desconto + Total de impostos + Frete  
   Frete = R$ 48,00  
   Total a pagar = 1.434,12 + 281,80 + 48,00 = R$ 1.763,92  

Agora, escrevendo a resposta final:

Answer: 1763.92

----

## Preparando para fazer o Self-Consistency

### Expressão regular para recuperar a resposta da última chamada

In [8]:
def extract_answer(text: str) -> str | None:
    m = re.search(r"(?i)answer\s*:\s*R?\$?\s*([0-9]+(?:[,.][0-9]{2}))", text)
    if m:
        # normaliza vírgula→ponto
        return m.group(1).replace(",", ".")
    return None

s = extract_answer(response.content)
print(s)

1763.92


In [9]:
type(s)

str

### Agora vou chamar o modelo 25 vezes e obter as respostas

In [10]:
lista_respostas = []
max_call = 25

# Loop para coletar 25 cadeias de raciocínio (k = 25)
for i in range(max_call):
    print(f"Chamada {i} de {max_call}")
    
    # 1. Chamar o modelo LLM com o prompt definido
    response = llm.invoke(prompt)        # Retorna um objeto com o texto gerado
    
    # 2. Extrair apenas a resposta final do texto completo
    s = extract_answer(response.content) # Função regex → "Answer: <valor>"
    
    # 3. Guardar a resposta extraída na lista
    lista_respostas.append(s)
    
    # 4. Esperar 10 s antes da próxima iteração
    # Ajuda a respeitar limites de rate-limit/custo
    time.sleep(10)                       

lista_respostas


Chamada 0 de 25
Chamada 1 de 25
Chamada 2 de 25
Chamada 3 de 25
Chamada 4 de 25
Chamada 5 de 25
Chamada 6 de 25
Chamada 7 de 25
Chamada 8 de 25
Chamada 9 de 25
Chamada 10 de 25
Chamada 11 de 25
Chamada 12 de 25
Chamada 13 de 25
Chamada 14 de 25
Chamada 15 de 25
Chamada 16 de 25
Chamada 17 de 25
Chamada 18 de 25
Chamada 19 de 25
Chamada 20 de 25
Chamada 21 de 25
Chamada 22 de 25
Chamada 23 de 25
Chamada 24 de 25


['1763.92',
 '1763.90',
 '1763.94',
 '1763.95',
 '1763.92',
 '1763.94',
 '1763.92',
 '1763.92',
 '1763.92',
 '1763.92',
 '1763.93',
 '1763.91',
 '1763.93',
 '1763.91',
 '1763.92',
 '1763.91',
 '1763.91',
 '1762.92',
 '1763.95',
 '1763.92',
 '1763.94',
 '1763.93',
 '1763.93',
 '1763.92',
 '1763.92']

### Medindo a frequência das respostas

In [11]:
# Medindo a frequência das respostas
counts = collections.Counter(lista_respostas)
counts

Counter({'1763.92': 10,
         '1763.93': 4,
         '1763.91': 4,
         '1763.94': 3,
         '1763.95': 2,
         '1763.90': 1,
         '1762.92': 1})

### Escolhendo o vencedor

In [16]:
counts.most_common(1)[0]

('1763.92', 10)

In [18]:
counts.most_common(1)[0][1] / 25

0.4